<a href="https://colab.research.google.com/github/smslca/Huggigfaces/blob/main/Finetuning_C11_HLLMBOOK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

In [2]:
from datasets import load_dataset
tomatoes = load_dataset('rotten_tomatoes')
train_data, test_data = tomatoes['train'], tomatoes['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_id = 'bert-base-cased'
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)
def preprocess(examples):
  return tokenizer(examples['text'],truncation=True)

tokenized_train = train_data.map(preprocess,batched=True)
tokenized_test = test_data.map(preprocess,batched=True)


In [23]:
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels,average=None)["f1"]
    return {"f1": f1}

In [24]:
from transformers import Trainer,TrainingArguments
training_args = TrainingArguments("model",
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=1,
                                  weight_decay=0.01,
                                  save_strategy='epoch',
                                  report_to="none")
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=tokenized_train,
                  eval_dataset=tokenized_test,
                  tokenizer=tokenizer,
                  data_collator = data_collator,
                  compute_metrics=compute_metrics,)

<ipython-input-24-5c0655436568>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


In [25]:
trainer.train()

Step,Training Loss
500,0.188700


TrainOutput(global_step=534, training_loss=0.2035490618216411, metrics={'train_runtime': 112.6549, 'train_samples_per_second': 75.718, 'train_steps_per_second': 4.74, 'total_flos': 227605451772240.0, 'train_loss': 0.2035490618216411, 'epoch': 1.0})

In [26]:
trainer.evaluate()

{'eval_loss': 0.4726119935512543,
 'eval_f1': array([0.84289746, 0.84377923]),
 'eval_runtime': 4.7659,
 'eval_samples_per_second': 223.67,
 'eval_steps_per_second': 14.058,
 'epoch': 1.0}